# Embeddings

Es gibt numerische Daten wie zum Beispiel Alter oder Absatzzahlen, die ein Machine-Learning-Modell direkt verarbeiten kann. Andere Daten sind dagegen kategorisch, wie zum Beispiel Wochentage oder Länder. Um mit diesen kategorischen Daten arbeiten zu können, nutzen wir Embeddings, die kategorische in numerische Werte verwandeln.

In diesem Notebook wird erklärt, wie man mithilfe der fastai-Bibliothek anhand eines Datensatzes von Covid-19-Infektionszahlen Embeddings entwickeln, tabellarische Daten voraussagen und kategorische Daten visualisieren kann. Anschließend gibt es die Möglichkeit, in der Übung dasselbe mit Absatzzahlen von des amerikanischen Supermarktes Walmart auszuprobieren.

### Einlesen der Daten
Zunächst werden die benötigten Bibliotheken numpy und pandas importiert. Wir importieren das tabular-Modul von fastai, das wir zur Verarbeitung von tabellarischen Daten verwenden.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import m3utils
from fastai.tabular.all import *

### Preprocessing
Wir nutzen wieder den Covid-Datensatz und verarbeiten ihn wie in den bereits vorangegangenen Einheiten. Neuronale Netze mögen normalisierte Daten, was wir mit ein paar zusätzlichen Transformationen unterstützen.

In [ ]:
df = m3utils.load_covid_19_data(m3utils.EU_UK_data()['Country/Region'])
df['value_trend_slope'] = df['value_trend_slope'] / (df['value_trend']+0.1)
df['value_trend'] = np.log(df['value_trend']+1)
df['value_daily'] = np.log(df['value_daily']+1)
df['Country/Region'] = df['Country/Region'].cat.remove_unused_categories() # damit wir später nicht für alle Länder ohne Daten Embeddings berechnen.
for lag in list(range(7, 0, -1)):
    df['value_daily_lag'+str(lag)] = df.groupby('Country/Region')['value_daily'].shift(lag)
for lag in list(range(3, 0, -1)):
    df['value_trend_lag'+str(lag)] = df.groupby('Country/Region')['value_trend'].shift(lag)
df.sample(5)

Wir möchten Infektionszahlen ('value_daily') vorhersagen und fügen als 'target_value' immer den Wert der nächsten Woche hinzu.

In [ ]:
df['target_value'] = df.groupby('Country/Region')['value_daily'].shift(-7).astype('float')
df.dropna(inplace=True)

Für die Validierung nehmen alle Daten ab April 2021 aus dem Trainingsdatensatz. Wir definieren, welche Spalten kategorisch und welche kontinuierlich (numerisch) sind.
Weiterhin definieren wir die genutzten Preprocessing-Funktionen. Categorify verändert die als kategorisch definierte Spalten in einen kategorischen Datentyp. FillMissing fügt fehlende Daten in den numerischen Variablen hinzu. Normalize wendet Standardisierung an. TabularPandas ist ein Container für tabellarische Trainings-Daten. Hier wird das zuvor spezifizierte Preprocessing vorgenommen.

In [ ]:
train_condition = df['date'] <= '2021-03-31'
train_idx = np.where(train_condition)[0]
valid_idx = np.where(~train_condition)[0]

splits = (list(train_idx),list(valid_idx))

cat_names = ['Country/Region', 'weekday']
cont_names = df.columns.drop(['Country/Region', 'weekday', 'date', 'target_value']).values.tolist()
preprocessing = [Categorify, FillMissing, Normalize]

to = TabularPandas(df, preprocessing, cat_names, cont_names, splits=splits, y_names='target_value')

In [ ]:
(len(to.train), len(to.valid))

### Trainingskonfiguration
Wir erstellen ein Neuronales-Netz mit dem `tabular_learner`. Wir definieren mit dem Parameter layers, wie viele Hidden-Layer das neuronale Netz, mit dem die Vorhersage gemacht wird, haben soll und wie viele Neuronen sich in jedem Layer befinden. Als Verlust-Funktion nehmen wir den Mean Squared Error. Mit dem Parameter `y_range` wird der Ausgabe-Bereich des Modells durch eine Sigmoid-Funktion angepasst.

In [ ]:
(df['target_value'].min(), df['target_value'].max())

In [ ]:
learn = tabular_learner(to.dataloaders(1024), layers=[200,100], y_range=(0, 13), loss_func=F.mse_loss, n_out=1)

Als Learning-Rate nehmen wir `0.033`.

In [ ]:
learn.fit_one_cycle(10, 0.033)

Wir wollen in diesem Notebook zeigen, wie die Embeddings trainiert werden und nicht, wie wir gute Vorhersagen mit neuronalen Netzen machen, deshalb geht es hier nach dem Training nicht mit den Vorhersagen weiter.

Wenn ihr noch mehr Interesse am Thema Forecasting und neuronale Netzwerke habt empfehlen wir euch das Projekt [GluonTS](https://ts.gluon.ai/) von Amazon oder für alle experimentierfreudigen auch [Pytorch-Forecasting](https://github.com/jdb78/pytorch-forecasting).

### Visualisierung
Nun möchten wir die Embeddings visualisieren. Dabei bietet sich die Country/Region an. Ob die Länder sich in der Visualisierung entsprechend ihrer geografischen Lage anordnen?

Wie sieht jetzt eins der gelernten Embeddings aus?

In [ ]:
next(learn.model.embeds[0].parameters())[0]

Wir können jetzt die Embeddings mit einer PCA in einen zwei dimensionalen Raum projezieren. Mithilfe einer Embeddings-Matrix werden die Koordinaten der Embeddings ermittelt, dementsprechend wie sie am Ende visualisiert werden sollen.

In [ ]:
from sklearn.decomposition import PCA

var = 'Country/Region'
var_index = cat_names.index(var)
var_values = list(df[var].astype('category').cat.categories.values)

emb_matrix = to_np(next(learn.model.embeds[var_index].parameters()))
emb_coordinates = PCA(n_components=2).fit_transform(emb_matrix)
annotation = np.append('Other', np.array(var_values))

emb_df = pd.DataFrame(emb_coordinates, columns=['X', 'Y'])
emb_df[var]=annotation

In [ ]:
fig = px.scatter(emb_df, x='X', y='Y', text='Country/Region')
fig.update_traces(textposition='top center')
fig.show()

## 1. Aufgabe - Embeddings für den Walmart-Datensatz
Nun sollst du dich am Walmart-Datensatz versuchen. Wir haben ein paar Features vorbereitet. Stelle eine Vorhersage der Absatzzahlen (`target_value`) und visualisiere z.B. die `store_id` oder `item_id`. Du kannst das Training z.B. bis 2015 machen (`df['wm_yr_wk']<1500`). Den Datensatz haben wir für dich schon vorbereitet.

In [ ]:
df = pd.read_csv('../input/m5-forecasting-parquet-and-aggregations/weekly_sales_items_top105.csv',
                dtype={'state_id': 'category', 'store_id': 'category', 'cat_id': 'category', 'dept_id': 'category', 'item_id': 'category'})
df['wm_yr_wk'] = df['wm_yr_wk']-10000
df['id'] = (df['store_id'].astype('str') + '_' + df['item_id'].astype('str')).astype('category')
df['value'] = np.log(df['value']+1)
df['value_trend'] = df.groupby(['id'], observed=True)['value'].rolling(window=4).mean().reset_index(level=0, drop=True)
df['value_trend_slope'] = df.groupby('id')['value_trend'].diff(4)
for lag in list(range(4, 0, -1)):
    df['value_lag'+str(lag)] = df.groupby('id')['value'].shift(lag)
for lag in list(range(4, 0, -1)):
    df['value_trend_lag'+str(lag)] = df.groupby('id')['value_trend'].shift(lag)

df['target_value'] = df.groupby('id')['value'].shift(-1).astype('float')

df.dropna(inplace=True)
df.drop(columns=['id'], inplace=True)

df.sample(5)

In [ ]:
# hier ein TabularPandas erstellen

In [ ]:
# hier das neuronale Netzwerk (tabular_learner) erstellen (vorher noch min/max für y_range checken)

In [ ]:
# trainieren (fit_one_cycle)

In [ ]:
# visualisieren

## Musterlösung: 1. Aufgabe - Embeddings für den Walmart-Datensatz


In [ ]:
train_condition = df['wm_yr_wk'] < 1500
train_idx = np.where(train_condition)[0]
valid_idx = np.where(~train_condition)[0]

splits = (list(train_idx),list(valid_idx))

cat_names = ['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']
cont_names = df.columns.drop(['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'wm_yr_wk']).values.tolist()
preprocessing = [Categorify, FillMissing, Normalize]

to = TabularPandas(df, preprocessing, cat_names, cont_names, splits=splits, y_names='target_value')

In [ ]:
(df['target_value'].min(), df['target_value'].max())

In [ ]:
learn = tabular_learner(to.dataloaders(1024), layers=[200,100], y_range=(0, 9), loss_func=F.mse_loss, n_out=1)

In [ ]:
learn.fit_one_cycle(10, 0.033)

In [ ]:
from sklearn.decomposition import PCA

var = 'store_id'
var_index = cat_names.index(var)
var_values = list(df[var].astype('category').cat.categories.values)

emb_matrix = to_np(next(learn.model.embeds[var_index].parameters()))
emb_coordinates = PCA(n_components=2).fit_transform(emb_matrix)
annotation = np.append('Other', np.array(var_values))

emb_df = pd.DataFrame(emb_coordinates, columns=['X', 'Y'])
emb_df[var]=annotation

fig = px.scatter(emb_df, x='X', y='Y', text=var)
fig.update_traces(textposition='top center')
fig.show()